# PART 3

#### Libraries

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albayerga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
from numpy import linalg as la
import matplotlib.pyplot as plt
import re
import string

#### Load and prepare some data

In [3]:
# original_data_tweets = pd.read_json('data/farmers-protest-tweets.json.gz', lines=True, compression='gzip')

In [4]:
tweet_document_ids_map =  pd.read_csv("data/tweet_document_ids_map.csv")
tweet_document_ids_map.head(5)

,docId,id
0,doc_0,1364506249291784198
1,doc_1,1364506237451313155
2,doc_2,1364506167226032128
3,doc_3,1364505991887347714
4,doc_4,1364505813834989568


In [5]:
document_id_map = dict(zip(tweet_document_ids_map['id'], tweet_document_ids_map['docId']))
for key in list(document_id_map.keys())[:5]:
    print(key, ":", document_id_map[key])

1364506249291784198 : doc_0
1364506237451313155 : doc_1
1364506167226032128 : doc_2
1364505991887347714 : doc_3
1364505813834989568 : doc_4


In [6]:
inverse_document_id_map = dict(zip(tweet_document_ids_map['docId'], tweet_document_ids_map['id']))
for key in list(inverse_document_id_map.keys())[:5]:
    print(key, ":", inverse_document_id_map[key])

doc_0 : 1364506249291784198
doc_1 : 1364506237451313155
doc_2 : 1364506167226032128
doc_3 : 1364505991887347714
doc_4 : 1364505813834989568


In [7]:
data_tweets = pd.read_csv('data/data_tweets.csv.gz', compression='gzip')
data_tweets = data_tweets[data_tweets['Language'] == 'en'].reset_index(drop=True)
data_tweets.head()

,Url,Date,Tweet,renderedContent,id,user,outlinks,tcooutlinks,replyCount,Retweets,...,conversationId,Language,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers,Hashtags
0,https://twitter.com/ArjunSinghPanam/status/136...,2021-02-24 09:23:35+00:00,The world progresses while the Indian police a...,The world progresses while the Indian police a...,1364506249291784198,"{'username': 'ArjunSinghPanam', 'displayname':...",['https://twitter.com/ravisinghka/status/13641...,['https://t.co/es3kn0IQAF'],0,0,...,1364506249291784198,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,NaN,NaN,{'url': 'https://twitter.com/RaviSinghKA/statu...,"[{'username': 'narendramodi', 'displayname': '...","['#ModiDontSellFarmers', '#FarmersProtest', '#..."
1,https://twitter.com/PrdeepNain/status/13645062...,2021-02-24 09:23:32+00:00,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,#FarmersProtest \n#ModiIgnoringFarmersDeaths \...,1364506237451313155,"{'username': 'PrdeepNain', 'displayname': 'Pra...",[],[],0,0,...,1364506237451313155,en,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,NaN,"[{'username': 'Kisanektamorcha', 'displayname'...","['#FarmersProtest', '#ModiIgnoringFarmersDeath..."
2,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:23:16+00:00,@ReallySwara @rohini_sgh watch full video here...,@ReallySwara @rohini_sgh watch full video here...,1364506167226032128,"{'username': 'anmoldhaliwal', 'displayname': '...",['https://youtu.be/-bUKumwq-J8'],['https://t.co/wBPNdJdB0n'],0,0,...,1364350947099484160,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,NaN,"[{'username': 'ReallySwara', 'displayname': 'S...","['#farmersprotest', '#NoFarmersNoFood']"
3,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:22:34+00:00,@mandeeppunia1 watch full video here https://t...,@mandeeppunia1 watch full video here youtu.be/...,1364505991887347714,"{'username': 'anmoldhaliwal', 'displayname': '...",['https://youtu.be/-bUKumwq-J8'],['https://t.co/wBPNdJdB0n'],0,0,...,1364428985074032646,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,NaN,"[{'username': 'mandeeppunia1', 'displayname': ...","['#farmersprotest', '#NoFarmersNoFood']"
4,https://twitter.com/anmoldhaliwal/status/13645...,2021-02-24 09:21:51+00:00,@mandeeppunia1 watch full video here https://t...,@mandeeppunia1 watch full video here youtu.be/...,1364505813834989568,"{'username': 'anmoldhaliwal', 'displayname': '...",['https://youtu.be/-bUKumwq-J8'],['https://t.co/wBPNdJdB0n'],0,0,...,1364480983995584515,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'thumbnailUrl': 'https://pbs.twimg.com/ext_t...,NaN,NaN,"[{'username': 'mandeeppunia1', 'displayname': ...","['#farmersprotest', '#NoFarmersNoFood']"


In [8]:
english_tweets_ids = data_tweets[data_tweets['Language'] == 'en']['id'].tolist()

#### Load inverted index (dictionary) from pickle file

We saved the inverted index in **index.pkl** to make it quicker to access later.

We also saved other data as dictionaries: **language_dict**, **tf**, **idf**, **df** and **docs**.

In [26]:
import pickle

# load index dictionary
with open('data/index.pkl', 'rb') as file:
    index_dictionary = pickle.load(file)

# load language dictionary
with open("data/language_dict.pkl", "rb") as file:
    language_dict = pickle.load(file)

# load tf, df and idf dictionaries
with open('data/tf.pkl', 'rb') as file:
    tf = pickle.load(file)

with open('data/df.pkl', 'rb') as file:
    df = pickle.load(file)

with open('data/idf.pkl', 'rb') as file:
    idf = pickle.load(file)

# load docs ({doc_id: [tokens]})
with open('data/docs.pkl', 'rb') as file:
    docs = pickle.load(file)

See how these dictionaries look like:

In [10]:
for term, postings in list(index_dictionary.items())[:5]:
    print(term, postings)

world [['doc_0', array('I', [0])], ['doc_38', array('I', [11])], ['doc_52', array('I', [7])], ['doc_86', array('I', [6])], ['doc_111', array('I', [8])], ['doc_143', array('I', [0])], ['doc_187', array('I', [3])], ['doc_248', array('I', [16])], ['doc_252', array('I', [4])], ['doc_336', array('I', [9])], ['doc_394', array('I', [13, 15])], ['doc_460', array('I', [3, 4])], ['doc_508', array('I', [7])], ['doc_519', array('I', [22])], ['doc_689', array('I', [5])], ['doc_751', array('I', [11])], ['doc_913', array('I', [0])], ['doc_926', array('I', [5])], ['doc_997', array('I', [15])], ['doc_1057', array('I', [10])], ['doc_1059', array('I', [4])], ['doc_1066', array('I', [7])], ['doc_1093', array('I', [9])], ['doc_1159', array('I', [11])], ['doc_1262', array('I', [12])], ['doc_1346', array('I', [9])], ['doc_1377', array('I', [1])], ['doc_1451', array('I', [7])], ['doc_1479', array('I', [2])], ['doc_1502', array('I', [2])], ['doc_1538', array('I', [0])], ['doc_1563', array('I', [1])], ['doc_157

In [27]:
for term, value in list(tf.items())[:1]:
    print(term, value) # term [[docId, tf], [docId, tf], ...]

for term, value in list(df.items())[:1]:
    print(term, value) # term df

for term, value in list(idf.items())[:1]:
    print(term, value) # term idf

for doc_id, tokens in list(docs.items())[:1]:
    print(doc_id, tokens)

world [0.1925, 0.2236, 0.1601, 0.1961, 0.1443, 0.1644, 0.2887, 0.1857, 0.2182, 0.1961, 0.3086, 0.417, 0.2182, 0.1302, 0.1525, 0.2673, 0.2357, 0.2085, 0.25, 0.2582, 0.2085, 0.189, 0.1741, 0.1826, 0.1796, 0.2041, 0.1857, 0.1768, 0.4472, 0.5, 0.2673, 0.4082, 0.3536, 0.1857, 0.2182, 0.2085, 0.2182, 0.2887, 0.169, 0.2294, 0.1925, 0.2182, 0.2085, 0.3536, 0.2, 0.1459, 0.2425, 0.2132, 0.2182, 0.4082, 0.25, 0.2673, 0.1961, 0.2294, 0.3162, 0.1313, 0.2182, 0.1667, 0.1525, 0.1525, 0.3015, 0.378, 0.3162, 0.2236, 0.1961, 0.1622, 0.2887, 0.2673, 0.189, 0.2085, 0.2582, 0.1429, 0.25, 0.3536, 0.1622, 0.2673, 0.5, 0.4082, 0.4472, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.2357, 0.3162, 0.2673, 0.3162, 0.2887, 0.2887, 0.2887, 0.2887, 0.2887, 0.2887, 0.2887, 0.3015, 0.2425, 0.1601, 0.1961, 0.1768, 0.2294, 0.3162, 0.3162, 0.2294, 0.2673, 0.2673, 0.2673, 0.2425, 0.2673, 0.1644, 0.1925, 0.1826, 0.2673, 0.2132, 0.1826, 0.1826, 0.1826, 0.1826, 0.1826, 0.1826, 0.182

#### Load necessary functions

In [19]:
def build_terms(line, lang):

    stemmer = PorterStemmer()
    stop_words = set(stopwords.words(lang)) # get the stop words for the language

    line = re.sub(r'http\S+', '', line)
    words_line= line.split() # tokenize the text, get a list of terms

    #First we deal with # separation
    treated_words = []
    for word in words_line:

        if word and word[0] == "#":  #If its a hashtag
            separated_list = re.split(r'(?<=[a-z])(?=[A-Z])', word[1:])
            for separated_word in separated_list:
                treated_words.append(separated_word)
        else:
            treated_words.append(word)


    line = [word.lower() for word in treated_words] # everything to lowercase
    translator = str.maketrans('', '', string.punctuation)
    line = [word.translate(translator) for word in line]  # remove punctuation

    line= [word for word in line if word not in stop_words] # remove stop_words
    line= [stemmer.stem(word) for word in line ] # steam
    line = [word for word in line if word.isalnum()]  # keeps only words with alphanumeric characters

    return line

### SCORE

In [13]:
query = "farmers protest"

#### TF-IDF + cosine similarity

In [ ]:
def rank_documents(terms, docs, index, idf, tf):

    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    # Compute the norm for the query tf
    query_terms_count = Counter(terms)  # get the frequency of each term in the query.
    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        # Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, doc_positions) in enumerate(index[term]):
            # Example of [doc_index, (doc, postings)]
            # 0 (26, array('I', [1, 4, 12, 15, 22, 28, 32, 43, 51, 68, 333, 337]))
            # 1 (33, array('I', [26, 33, 57, 71, 87, 104, 109]))
            # term is in doc 26 in positions 1,4, .....
            # term is in doc 33 in positions 26,33, .....

            # tf[term][0] will contain the tf of the term "term" in the doc 26
            if doc in docs:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]

    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)

    result_docs = [x[1] for x in doc_scores]

    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)

    return result_docs


def search_tf_idf(query, index):
    """
    output is the list of documents that contain any of the query terms.
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query,'english')
    docs = set()
    i=0
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"
            term_docs = [posting[0] for posting in index[term]]

            if i == 0:
                docs = set(term_docs)
                i = 1
            # docs = docs intersection term_docs
            else: docs &= set(term_docs)

        except:
            # term is not in index
            pass

    docs = list(docs)

    ranked_docs = rank_documents(query, docs, index, idf, tf)

    # print( ranked_docs)
    return ranked_docs

In [25]:
# return the top 5 docs that contain the query terms and its score
search_tf_idf(query, index_dictionary)[:5]

['doc_32542', 'doc_22865', 'doc_7859', 'doc_6060', 'doc_5727']

In [33]:
# see docs obtained from the search in docs
doc_ids = ['doc_32542', 'doc_22865', 'doc_7859', 'doc_6060', 'doc_5727']
for doc_id in doc_ids:
    print(docs[doc_id])

['timesnow', 'farmer', 'protest', 'protest', 'protest']
['largest', 'protest', 'kanganateam', 'farmer', 'protest', 'farmer', 'protest', 'farmer', 'protest']
['farmer', 'protest', 'farmer', 'protest']
['farmer', 'protest']
['farmer', 'protest', 'farmer', 'protest']


#### Your-Score + cosine similarity

In [ ]:
# code here

#### BM25

In [31]:
from collections import defaultdict, Counter
import numpy as np

def rank_documents_bm25(terms, docs, index, idf, tf, doc_lengths, avg_doc_length, k_1=1.5, b=0.75):
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    # Compute the norm for the query tf
    query_terms_count = Counter(terms)  # get the frequency of each term in the query.
    query_norm = np.linalg.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):
        if term not in index:
            continue

        # Compute BM25 for the query (normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf.get(term, 0)

        # Generate doc_vectors for matching docs
        for doc_index, (doc, doc_positions) in enumerate(index[term]):
            if doc in docs:
                # TF in BM25 formula
                tf_value = tf[term][doc_index]
                doc_len = doc_lengths[doc]  # Length of the document
                bm25_weight = idf.get(term, 0) * (tf_value * (k_1 + 1)) / (tf_value + k_1 * (1 - b + b * doc_len / avg_doc_length))
                doc_vectors[doc][termIndex] = bm25_weight

    # Compute scores using dot product
    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)

    result_docs = [x[1] for x in doc_scores]

    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_bm25(query, index)

    return result_docs


def search_bm25(query, index, idf, tf, doc_lengths, avg_doc_length):
    """
    output is the list of documents that contain any of the query terms.
    So, we will get the list of documents for each query term, and take the union of them.
    """
    query = build_terms(query, 'english')
    docs = set()
    for term in query:
        try:
            term_docs = [posting[0] for posting in index[term]]
            docs.update(term_docs)
        except KeyError:
            pass

    docs = list(docs)
    ranked_docs = rank_documents_bm25(query, docs, index, idf, tf, doc_lengths, avg_doc_length)

    return ranked_docs


In [29]:
doc_lengths = {doc_id: len(doc_terms) for doc_id, doc_terms in docs.items()}
average_doc_length = sum(doc_lengths.values()) / len(doc_lengths)
print("Average document length:", average_doc_length)

Average document length: 16.673067789960562


In [32]:
search_bm25(query, index_dictionary, idf, tf, doc_lengths, average_doc_length)[:5]

['doc_6060', 'doc_5183', 'doc_46137', 'doc_45189', 'doc_43199']

In [34]:
doc_ids = ['doc_6060', 'doc_5183', 'doc_46137', 'doc_45189', 'doc_43199']
for doc_id in doc_ids:
    print(docs[doc_id])

['farmer', 'protest']
['farmer', 'protest']
['farmer', 'protest']
['farmer', 'protest']
['farmer', 'protest']
